Существует несколько популярных библиотек, упрощающих разработку рекомендательных систем. В Sklearn к сожалению нет готовых методов для генерации рекомендаций (считается, что их можно получить базовым функционалом библиотеки).

В этом упражнении рассомтрим три популярные билиотеки: **lightFM**, **Surprise** и **Implicit**



# <span style="color:blue">LightFM<span>

Установка:

    pip install lightfm
    
или
    
    conda install -c conda-forge lightfm

или
    
    через GUI анаконды (проверить, что добавлен channel "conda-forge")

В библиотеке есть 2 встроенных датасета: 
- Movielens (оценки фильмов по шкале 1-5)
- Stackexchange

Будем работать с MovieLens:

In [129]:
from lightfm.datasets import fetch_movielens
data = fetch_movielens()

titles = data['item_labels']
ratings = data['train'].tocsr()
ratings_test = data['test'].tocsr()

Данные выгружаются в разреженном формате COO, поэтому для удобства можем перевести в формат CSR (для этого есть метод tocsr()).


Проставленные оценки пользователя

In [125]:
pandas.Series(ratings[0,:].toarray()[0]).value_counts()

0    1420
5      79
4      77
3      54
2      27
1      25
dtype: int64

In [83]:
data['test']

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>

Создание модели сводится к нескольким строкам.



In [132]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

model = LightFM(loss='warp')
model.fit(ratings, epochs=30, num_threads=2)
test_precision = precision_at_k(model, ratings_test, k=5).mean()

Попробуем вручную загрузить данные

В качестве основного параметра мы передали матрицу оценок. В этом случае выполняется матричная факторизация (matrix factorization).

In [131]:
test_precision

0.11049842

# <span style="color:blue">Surprise</span>

Вторая библиотека, которую мы рассмотрим, называется **Surprise**.

Варианты усатновки:
- pip install scikit-learn
- conda -c conda-forge install scikit-surprise
- через Anaconda GUI 

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

В бибилиотеке Surprise датасет также можно подгрузить из кода. Для этого есть Dataset.load_builtin

In [ ]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# We'll use the famous SVD algorithm.
algo = SVD()

In [ ]:
for x in data.folds():
    print(x[0])

In [ ]:
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

# <span style="color:blue">Implicit<span>

**Implicit** - библиотека, заточенная под работу с неяынми рейтингами.

- pip install implicit
- conda install -c conda-forge implicit
- Anaconda GUI

In [1]:
import implicit

import os
import pandas
data_path = "/Users/Konstantin/HSE/MasterProgram/Практический Семинар/Recommender Systems/RecSys_lesson1/ml-1m/"

Загрузка тестового датасета выполняется методом **get_movielens()**. Параметр variant определяет размер загружаемой выборки. Мы будем использовать "1m" (1 млн оценок).

На выходе:
- ratings - матрица оценок (в sparse-формате)
- titles - названия товаров (фильмы)

In [37]:
from implicit.datasets.movielens import get_movielens
titles, ratings = get_movielens(variant='1m')

Приведем матрицу оценок к бинарному формату 0/1

In [39]:
import numpy
ratings.data[ratings.data < 4.0] = 0
ratings.eliminate_zeros()
ratings.data = numpy.ones(len(ratings.data))

In [18]:
ratings.shape

(3953, 6041)

В implicit есть несколько моделей:
- AlternatingLeastSquares
- BayesianPersonalizedRanking
- TFIDFRecommender
- CosineRecommender
- BM25Recommender

In [62]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(ratings)

100%|██████████| 15.0/15 [00:04<00:00,  2.82it/s]


Возьмем какого-нибудь юзера и посмотрим, какие фильмы ему понравились

In [58]:
userid = 4
for y in [titles[x] for x in numpy.nonzero(ratings.T[userid,:])[1]]:
    print(y)

Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993)
Die Hard (1988)
E.T. the Extra-Terrestrial (1982)
Raiders of the Lost Ark (1981)
Good, The Bad and The Ugly, The (1966)
Alien (1979)
Terminator, The (1984)
Jaws (1975)
Rocky (1976)
Saving Private Ryan (1998)
King Kong (1933)
Run Lola Run (Lola rennt) (1998)
Goldfinger (1964)
Fistful of Dollars, A (1964)
Thelma & Louise (1991)
Hustler, The (1961)
Mad Max (1979)


Сгенерировать реокемндации - model.recommend()

In [66]:
recommended_film_ids = model.recommend(userid=userid, user_items=ratings.T.tocsr(), N=5)
recommended_film_ids

[(589, 0.4580247),
 (2571, 0.4282091),
 (1291, 0.40621763),
 (1304, 0.38326198),
 (1196, 0.37946856)]

Переведем айдишники в названия

In [70]:
recommened_film_ids = [(titles[x[0]],x[1]) for x in recommended_film_ids]
recommened_film_ids

[('Terminator 2: Judgment Day (1991)', 0.4580247),
 ('Matrix, The (1999)', 0.4282091),
 ('Indiana Jones and the Last Crusade (1989)', 0.40621763),
 ('Butch Cassidy and the Sundance Kid (1969)', 0.38326198),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 0.37946856)]

In [75]:
numpy.ediff1d(ratings.indptr)

array([   0, 1655,  285, ...,   33,   27,  264], dtype=int32)